# Kernel PCA

 1. The traditional PCA and the LDA algorithms works pretty good with the data that is linearly seperable but fails   to work in case when the data points are not linearly seperable.
<img src="./images/graph_1.png" height="50%" width="50%">

 2. This is when **Kernel PCA** comes handy(which relates to the concept of kernel SVM).
 3. We are performing a non-linear mapping via Kernel PCA of the data to higher dimensional space. We then use traditional PCA to transform the data into lower dimension where the data points can be linearly seperated.
 4. One downside of this approach is that it is computationally very expensive, thus we use **kernel tricks**, that let's us explore the similarities between the 2 higher dimensional features in its original feature space.
 5. A **Kernel Function** can be understood as a function that calculated dot product between 2 vectors - i.e. the measure of similarity.
 6. Different Kernel Function encompass :-
   * Polynomial Kernel : $k(x^{i},x^{j})=(x^{(i)^T}x^{(j)}+\theta)^p$ , where $\theta$ is the threshold and p is the power
   * Hyperbolic tangent
   * Radial Bias Function(RBF) or **Gaussian Kernel** : $k(x^{i},x^{j})=exp\bigl(-\frac{||x^{(i)}-x^{(j)}||^2}{2\sigma^2}\bigr)$, where $\gamma=\frac{1}{2\sigma^2}$
 7. **Steps involved** :
   1. We first compute the **kernel similarity matrix K**, where we compute the dot product for each and every pair of data points.
   $$K=
    \begin{pmatrix}
    k(x^{(1)},x^{(1)}) & k(x^{(1)},x^{(2)}) & \cdots & k(x^{(1)},x^{(n)}) \\
    k(x^{(2)},x^{(1)}) & k(x^{(2)},x^{(2)}) & \cdots & k(x^{(2)},x^{(n)}) \\
    \vdots & \vdots & \ddots & \vdots \\
    k(x^{(n)},x^{(1)}) & k(x^{(n)},x^{(2)}) & \cdots & k(x^{(n)},x^{(n)})
    \end{pmatrix}
$$

  2. In PCA we standardize the data such that the mean for the data points is 0, in this case as we project the data onto higher dimension there is no assurance that the data points will be standardize, hence its important to standardize the data. $K=K - 1_nK - K1_n - 1_nK1_n$ , where $1_n$ is the n x n matrix with values 1/n.
  3. We collect the top k eigenvectors of the centered kernel matrix based on their corresponding eigenvalues, which are ranked by decreasing magnitude. In contrast to standard PCA, the eigenvectors are not the principal component axes, but the samples already projected onto these axes.

In [1]:
from scipy.spatial.distance import pdist,squareform
from scipy import exp
from scipy.linalg import eigh
import numpy as np

In [50]:
def rbf_kernel_pca(X,gamma,n_components):
    """
    RBF kernel PCA implementation.
    Parameters
    ------------
    X: {NumPy ndarray}, shape = [n_samples, n_features]
        gamma: float
          Tuning parameter of the RBF kernel
        n_components: int
          Number of principal components to return
    Returns ------------
    X_pc: {NumPy ndarray}, shape = [n_samples, k_features] Projected dataset

    """
    # Calculate the pairwise squared Euclidean distances in the MxN dimensional dataset
    # implements : sum((x[0]-x[1])**2)
    sq_dist=pdist(X,'sqeuclidean')
    
    
    # Converts pariwise distances into a square martix.
    # i.e. distance x[i]-x[j] will be placed at position (i,j)
    mat_sq_dists=squareform(sq_dist)
    
    # Compute the symmetric kernel matrix
    K=np.exp(-gamma*mat_sq_dists)
    
    # Standardizing the new data points
    N=K.shape[0]
    
    one_n=np.ones((N,N))/N
    K=K - one_n.dot(K) - K.dot(one_n) - (one_n.dot(K)).dot(one_n)
    
    # obtaining eigen pairs from the centered kernel matrix
    # Diff between eig and eigh is that eigh return sorted eigen values in ascending order
    # and should be used in case when the input matrix is symmetric
    eigen_vals,eigen_vecs=np.linalg.eigh(K)
    
    # Rearranging in descending order
    eigen_vals,eigen_vecs=eigen_vals[::-1],eigen_vecs[:,::-1]
    
    # Collect top k eigen vectors
    X_pc=np.column_stack((eigen_vecs[:,i] for i in range(n_components)))
    
    return X_pc

The value of $\gamma$ needs to be give in advance and the best way to choose the value for $\gamma$ is via **HyperParameter tunning Algorithms**